In [2]:
import pandas as pd

# Load the dataset
file_path = "/content/combined_sorted_data_2022_2023.csv"
df = pd.read_csv(file_path)

# Remove data for "Central India - Achanakmar Sanctuary"
df = df[df['region'] != "Central India - Achanakmar Sanctuary"]

# Save the filtered dataset
df.to_csv("Filtered_Dataset.csv", index=False)

print("Filtered dataset saved as 'Filtered_Dataset.csv'.")


<ipython-input-2-ef578dd5a28b>:5: DtypeWarning: Columns (10,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Filtered dataset saved as 'Filtered_Dataset.csv'.


In [4]:
import pandas as pd

# Load the dataset
file_path = "/content/Filtered_Dataset.csv"
df = pd.read_csv(file_path)

# Convert 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'], dayfirst=True)

# Count occurrences of each (latitude, longitude) pair
location_counts = df.groupby(['latitude', 'longitude']).size()

# Keep only locations with at least 10 records
valid_locations = location_counts[location_counts >= 10].index
df_filtered = df[df.set_index(['latitude', 'longitude']).index.isin(valid_locations)]

# Create a complete date range for each (latitude, longitude) pair
upsampled_data = []

for (lat, lon), group in df_filtered.groupby(['latitude', 'longitude']):
    # Remove duplicate dates
    group = group.drop_duplicates(subset=['date'])

    # Create a full date range
    full_date_range = pd.date_range(start=group['date'].min(), end=group['date'].max(), freq='D')

    # Set index and reindex
    group = group.set_index('date').reindex(full_date_range).interpolate().reset_index()

    # Rename the date column back
    group.rename(columns={'index': 'date'}, inplace=True)

    # Assign latitude and longitude
    group[['latitude', 'longitude']] = lat, lon

    upsampled_data.append(group)

# Concatenate all upsampled data
df_upsampled = pd.concat(upsampled_data, ignore_index=True)

# Correct the year, month, and day columns
df_upsampled['year'] = df_upsampled['date'].dt.year
df_upsampled['month'] = df_upsampled['date'].dt.month
df_upsampled['day'] = df_upsampled['date'].dt.day

# Assign the 'region' column
df_upsampled['region'] = "Central India - Satpura Range"

# Save the final dataset
df_upsampled.to_csv("Satpura_Range_Upsampled_Final.csv", index=False)

print("Upsampling completed and saved as 'Satpura_Range_Upsampled_Final.csv'")


<ipython-input-4-7ab3fd835f35>:5: DtypeWarning: Columns (10,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
<ipython-input-4-7ab3fd835f35>:28: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  group = group.set_index('date').reindex(full_date_range).interpolate().reset_index()
<ipython-input-4-7ab3fd835f35>:28: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  group = group.set_index('date').reindex(full_date_range).interpolate().reset_index()
<ipython-input-4-7ab3fd835f35>:28: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  group = group.set_index('date').reindex(f

Upsampling completed and saved as 'Satpura_Range_Upsampled_Final.csv'


In [6]:
import pandas as pd
import os

merged_file = "Satpura_Range_Upsampled_Final.csv"

try:
    # Verify file exists
    if not os.path.exists(merged_file):
        raise FileNotFoundError(f"Merge file {merged_file} not found")

    # Load data with FRP values
    df = pd.read_csv(merged_file)

    # Verify FRP column exists
    if 'frp' not in df.columns:
        raise KeyError("FRP column missing in merged data")

    # Calculate counts
    total_records = df.shape[0]
    zero_frp = (df['frp'] == 0).sum()
    non_zero_frp = total_records - zero_frp

    # Print formatted results
    print(f"Total records: {total_records:,}")
    print(f"Records with FRP = 0: {zero_frp:,} ({zero_frp/total_records:.1%})")
    print(f"Records with FRP > 0: {non_zero_frp:,} ({non_zero_frp/total_records:.1%})")

except Exception as e:
    print(f"Validation failed: {str(e)}")


Total records: 382,725
Records with FRP = 0: 382,412 (99.9%)
Records with FRP > 0: 313 (0.1%)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering, KMeans, DBSCAN
from sklearn.metrics import silhouette_score
import numpy as np

# Load dataset
file_path = "Satpura_Range_Upsampled_Final.csv"
df = pd.read_csv(file_path)

# Extract latitude and longitude
coords = df[['latitude', 'longitude']].to_numpy()

# List to store silhouette scores for comparison
silhouette_scores = {'Agglomerative': [], 'KMeans': [], 'DBSCAN': []}
cluster_counts = {'Agglomerative': [], 'KMeans': [], 'DBSCAN': []}

# Range of clusters to test for Agglomerative and KMeans
range_n_clusters = range(2, 11)  # Checking between 2 and 10 clusters for KMeans and Agglomerative

# Agglomerative Clustering
for n_clusters in range_n_clusters:
    clustering = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward').fit(coords)
    silhouette_scores['Agglomerative'].append(silhouette_score(coords, clustering.labels_))
    cluster_counts['Agglomerative'].append(n_clusters)

# KMeans Clustering
for n_clusters in range_n_clusters:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(coords)
    silhouette_scores['KMeans'].append(silhouette_score(coords, kmeans.labels_))
    cluster_counts['KMeans'].append(n_clusters)

# DBSCAN Clustering
# DBSCAN doesn't need the number of clusters, but we can tune 'eps' and 'min_samples'
eps_values = np.linspace(0.1, 1.5, 15)  # range of eps to try
min_samples_values = [3, 4, 5, 6]  # range of min_samples to try

best_silhouette = -1
best_eps = None
best_min_samples = None

for eps in eps_values:
    for min_samples in min_samples_values:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples).fit(coords)
        labels = dbscan.labels_

        # Check if the labels contain more than one cluster
        if len(set(labels)) > 1 and -1 not in labels:  # Exclude noise points (-1)
            silhouette_avg = silhouette_score(coords, labels)
            silhouette_scores['DBSCAN'].append(silhouette_avg)
            cluster_counts['DBSCAN'].append(f"eps={eps}, min_samples={min_samples}")

            if silhouette_avg > best_silhouette:
                best_silhouette = silhouette_avg
                best_eps = eps
                best_min_samples = min_samples

# Plot Silhouette Scores for each algorithm
plt.figure(figsize=(10, 6))

# Agglomerative Clustering Silhouette Score
plt.plot(cluster_counts['Agglomerative'], silhouette_scores['Agglomerative'], marker='o', label='Agglomerative Clustering')

# KMeans Clustering Silhouette Score
plt.plot(cluster_counts['KMeans'], silhouette_scores['KMeans'], marker='o', label='KMeans Clustering')

# DBSCAN Clustering Silhouette Score
plt.plot(cluster_counts['DBSCAN'], silhouette_scores['DBSCAN'], marker='o', label='DBSCAN Clustering')

plt.xlabel('Number of clusters / DBSCAN Params')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score Comparison for Agglomerative, KMeans, and DBSCAN Clustering')
plt.legend()
plt.xticks(rotation=90)
plt.show()

# Optimal number of clusters for Agglomerative and KMeans
optimal_agglomerative_clusters = cluster_counts['Agglomerative'][np.argmax(silhouette_scores['Agglomerative'])]
optimal_kmeans_clusters = cluster_counts['KMeans'][np.argmax(silhouette_scores['KMeans'])]

# Optimal DBSCAN parameters
optimal_dbscan_params = (best_eps, best_min_samples)

print(f"Optimal number of clusters for Agglomerative Clustering: {optimal_agglomerative_clusters}")
print(f"Optimal number of clusters for KMeans Clustering: {optimal_kmeans_clusters}")

# Get the number of clusters excluding noise (-1)
dbscan = DBSCAN(eps=0.8, min_samples=3)
labels = dbscan.fit_predict(coords)

# Count unique clusters (excluding noise)
unique_labels = set(labels) - {-1}
print(f"Number of clusters (excluding noise): {len(unique_labels)}")

# Get the number of clusters excluding noise (-1)
dbscan = DBSCAN(eps=0.8, min_samples=3)
labels = dbscan.fit_predict(coords)

# Count unique clusters (excluding noise)
unique_labels = set(labels) - {-1}
print(f"Optimal Number of clusters for DBSCAN:  {len(unique_labels)}")



In [1]:
import pandas as pd
from sklearn.cluster import KMeans

# Load dataset
file_path = "/content/Satpura_Range_Upsampled_Final.csv"
df = pd.read_csv(file_path)

# Extract latitude and longitude
coords = df[['latitude', 'longitude']].to_numpy()

# Apply K-Means Clustering to create 7 clusters
num_clusters = 7
kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
df['cluster'] = kmeans.fit_predict(coords)

# Save each cluster as a separate CSV file
cluster_files = []
for i in range(num_clusters):
    cluster_data = df[df['cluster'] == i]
    file_name = f"cluster_{i}.csv"
    cluster_data.to_csv(file_name, index=False)
    cluster_files.append(file_name)

cluster_files  # List of generated files


['cluster_0.csv',
 'cluster_1.csv',
 'cluster_2.csv',
 'cluster_3.csv',
 'cluster_4.csv',
 'cluster_5.csv',
 'cluster_6.csv']

In [2]:
import pandas as pd
import glob

# Load all cluster files
cluster_files = glob.glob("cluster_*.csv")
clusters_data = {file: pd.read_csv(file) for file in cluster_files}

# Validate clusters
validation_results = {}

for file, df in clusters_data.items():
    cluster_id = file.split("_")[-1].split(".")[0]
    unique_lat_long = df[['latitude', 'longitude']].drop_duplicates().shape[0]

    validation_results[cluster_id] = {
        "Total Points": len(df),
        "Unique Lat-Long Pairs": unique_lat_long,
        "Latitude Range": (df["latitude"].min(), df["latitude"].max()),
        "Longitude Range": (df["longitude"].min(), df["longitude"].max()),
        "FRP Range": (df["frp"].min(), df["frp"].max()),
    }

# Convert validation results to a DataFrame for better readability
validation_df = pd.DataFrame.from_dict(validation_results, orient="index")
validation_df


<ipython-input-2-e23e68ea3ece>:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  clusters_data = {file: pd.read_csv(file) for file in cluster_files}


,Total Points,Unique Lat-Long Pairs,Latitude Range,Longitude Range,FRP Range
1,45927,63,"(22.0, 24.25)","(78.5, 80.0)","(0.0, 42.8)"
6,59778,82,"(24.0, 26.0)","(75.0, 77.25)","(0.0, 25.5)"
2,60507,83,"(20.0, 21.75)","(75.0, 77.5)","(0.0, 23.1)"
4,60507,83,"(20.0, 22.0)","(77.75, 80.0)","(0.0, 47.18)"
5,45927,63,"(21.75, 24.0)","(75.0, 76.5)","(0.0, 15.34)"
3,60507,83,"(24.25, 26.0)","(77.5, 80.0)","(0.0, 13.3)"
0,49572,68,"(21.75, 24.25)","(76.75, 78.25)","(0.0, 96.5)"


LSTM Model Used for Weekly prediction fires

In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset

# Load multiple CSV files
file_paths = ["cluster_0.csv", "cluster_1.csv", "cluster_2.csv", "cluster_3.csv",
              "cluster_4.csv", "cluster_5.csv", "cluster_6.csv"]

# Read and concatenate all files
df_list = [pd.read_csv(file) for file in file_paths]
df = pd.concat(df_list, ignore_index=True)

# Ensure 'date' is in datetime format
df['date'] = pd.to_datetime(df['date'])

# Filter train dataset to only 2022 data
df_train = df[(df['date'] >= '2022-01-01') & (df['date'] <= '2022-12-31')]

# Filter actual dataset for the first week of 2023
df_actual = df[(df['date'] >= '2023-01-01') & (df['date'] <= '2023-01-07')]

# Set 'date' as index before resampling
df_train = df_train.set_index('date')
df_actual = df_actual.set_index('date')

# Resample data to weekly FRP
df_train_weekly = df_train.resample('W').mean(numeric_only=True).reset_index()
df_actual_weekly = df_actual.resample('W').mean(numeric_only=True).reset_index()

# Check for NaN or Infinite values and remove them
df_train_weekly = df_train_weekly.dropna()
df_actual_weekly = df_actual_weekly.dropna()
df_train_weekly = df_train_weekly[~np.isinf(df_train_weekly['frp'])]
df_actual_weekly = df_actual_weekly[~np.isinf(df_actual_weekly['frp'])]

# Apply Log Scaling before MinMaxScaler
df_train_weekly['frp'] = np.log1p(df_train_weekly['frp'])  # Log transform

# Normalize the FRP values
scaler = MinMaxScaler()
df_train_weekly['frp'] = scaler.fit_transform(df_train_weekly[['frp']])

# Convert Data to PyTorch Tensors
def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i+seq_length])
        labels.append(data[i+seq_length])
    return np.array(sequences), np.array(labels)

seq_length = 4  # 4 weeks as input
train_sequences, train_labels = create_sequences(df_train_weekly['frp'].values, seq_length)

# Convert to tensors
train_sequences = torch.tensor(train_sequences, dtype=torch.float32).unsqueeze(-1)
train_labels = torch.tensor(train_labels, dtype=torch.float32).unsqueeze(-1)

# LSTM Model
class LSTMModel(nn.Module):
    def __init__(self):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=64, num_layers=2, batch_first=True)
        self.fc = nn.Linear(64, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

# Initialize Model, Loss, and Optimizer
model = LSTMModel()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop with Gradient Clipping and NaN Check
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(train_sequences)
    loss = criterion(output, train_labels)

    # Check for NaN loss
    if torch.isnan(loss):
        print("NaN detected in loss, stopping training!")
        break

    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient Clipping
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# Predict for First Week of 2023
model.eval()
last_4_weeks = torch.tensor(df_train_weekly['frp'].values[-seq_length:], dtype=torch.float32).unsqueeze(0).unsqueeze(-1)
predicted_frp = model(last_4_weeks).item()

# Inverse Transform Prediction
predicted_frp = np.expm1(scaler.inverse_transform([[predicted_frp]])[0][0])  # Reverse Log Scaling

# Actual FRP for Comparison
actual_frp = df_actual_weekly['frp'].values[0] if len(df_actual_weekly) > 0 else "No Data"

print(f"Predicted FRP for the first week of January 2023: {predicted_frp}")
print(f"Actual FRP for the first week of January 2023: {actual_frp}")


<ipython-input-4-86f99233b986>:14: DtypeWarning:

Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.



Epoch 0, Loss: 0.0369
Epoch 10, Loss: 0.0351
Epoch 20, Loss: 0.0321
Epoch 30, Loss: 0.0290
Epoch 40, Loss: 0.0267
Epoch 50, Loss: 0.0232
Epoch 60, Loss: 0.0177
Epoch 70, Loss: 0.0135
Epoch 80, Loss: 0.0126
Epoch 90, Loss: 0.0123
Predicted FRP for the first week of January 2023: 0.008255771376502697
Actual FRP for the first week of January 2023: 0.009047619047619047
